In [3]:
!pip install pymilvus
!pip install pyPdf

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12

In [4]:

from pymilvus import MilvusClient, DataType
client = MilvusClient(uri="http://192.168.3.116:19530")

In [6]:
collection_name = "gpus"
embedding_dim=2048
if client.has_collection(collection_name=collection_name):
    client.drop_collection(collection_name=collection_name)

In [21]:

schema = client.create_schema(auto_id=False, enable_dynamic_field=False)
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=True,max_length=65535)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=embedding_dim)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=65535)
schema.add_field(field_name="metadata", datatype=DataType.VARCHAR, max_length=65535)

index_params = MilvusClient.prepare_index_params()
index_params.add_index(
    field_name="vector",
    metric_type="L2",
    index_type="AUTOINDEX",
)

if client.has_collection(collection_name=collection_name):
    client.drop_collection(collection_name=collection_name)
client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params,
    consistency_level="Strong",
)

client.load_collection(collection_name=collection_name)


In [16]:
from pypdf import PdfReader

def pdf_text(path):# pageは配列
    reader = PdfReader(path)
    #texts = [reader.pages[i].extract_text() or "" for i in page ]
    texts = [ j.extract_text() or "" for j in reader.pages]
    return " ".join(texts)

#def clean_cut_text(text):
#    text2 = text.replace("\n","")
#   return text2.split(".")

def clean_cut_text(text):
    text2 = text.replace("\n", "")
    clean_text_list = [s.strip() for s in text2.split(".") if s.strip()]
    sentence = ""
    for j in clean_text_list:
        sentence = sentence + j + " "
    return sentence

def extract_pdf(path):
    text = pdf_text(path)
    texts = clean_cut_text(text)
    return texts

def chunk_text_by_chars(text, name, chunk_size=512, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(f"{name} is {text[start:end]}")
        start += chunk_size - overlap
    return chunks





In [13]:
names = ["blackwell-rtx-pro-4000","blackwell-rtx-pro-5000","blackwell-rtx-pro-4500",
         "rtx-4500","rtx-2000-ada","a800-40gb","blackwell-rtx-pro-6000","rtx-4000-ada",
         "rtx-4000-sff-ada","rtx-5880-ada","blackwell-rtx-pro-6000-max-q","rtx6000"]

In [14]:
import os
#ファイル名と製品名を記入
PATH = "/workspace/Datasheet"
files = os.listdir(f"{PATH}/GPU/")

In [17]:
product_name = []
product_chunks = []
for i in range(len(files)):
    texts = extract_pdf(f"{PATH}/GPU/{files[i]}")
    chunks = chunk_text_by_chars(texts,names[i])
    product_chunks = product_chunks + chunks
    for j in range(len(chunks)):
        product_name.append(names[i])

In [18]:
import requests
import json

url = "http://llama-32-nv-embedqa-1b-v2.runai-clv01.aitrynow-run-inf.macnica.co.jp/v1/embeddings"

headers = {
    "Content-Type": "application/json"
}

def embedding(text):
    payload ={
    "input": text,
    "model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
    "input_type": "passage",
    "encoding_format": "float",
    "dimensions": None,
    "user": "user-identifier",
    "truncate": "NONE"
  }

    return requests.post(url, headers=headers, data=json.dumps(payload)).json()['data'][0]['embedding']
    

In [19]:
import numpy as np
vectors = []

for i in product_chunks:
    vectors.append(np.array(embedding(i)))
print("Dim:", vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": str(i), "vector": vectors[i], "text": product_chunks[i],"metadata": product_name[i]}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

Dim: (2048,)
Data has 152 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'metadata'])
Vector dim: 2048


In [22]:
res = client.insert(collection_name=collection_name, data=data)

print(res)

{'insert_count': 152, 'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151']}
